# Notebook for cleaning `PROCEDURES TABLE`

In [1]:
import pandas as pd
import numpy as np
 
from sqlalchemy import create_engine,text
user = 'root'
passwords = '7003890541'
host = 'localhost'
port = 3306
database = 'synthea'

engine = create_engine(f"mysql+pymysql://{user}:{passwords}@{host}:{port}/{database}")
def sql(query , engine = engine):
    return pd.read_sql(query,engine)


def select(table_name):
    return sql(f'select * from {table_name} limit 5')


def run(query):
    with engine.connect() as con:
        con.execute(text(query))
        con.commit()

In [2]:
select('procedures')

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
0,2011-04-30T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,169553002,Insertion of subcutaneous contraceptive,14896.56,NaN,None
1,2010-07-27T12:58:08Z,10339b10-3cd1-4ac3-ac13-ec26728cb592,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,430193006,Medication Reconciliation (procedure),726.51,NaN,None
2,2010-11-20T03:04:34Z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,7ff86631-0378-4bfc-92ce-1edd697eb18e,430193006,Medication Reconciliation (procedure),788.50,NaN,None
3,2011-02-07T03:04:34Z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,117015009,Throat culture (procedure),2070.44,195662009.0,Acute viral pharyngitis (disorder)
4,2011-04-19T03:04:34Z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,640837d9-845a-433c-9fad-47426664a69d,117015009,Throat culture (procedure),2479.39,195662009.0,Acute viral pharyngitis (disorder)


In [3]:
query = '''
    alter table procedures
    drop column code,
    drop column reasoncode;
'''
run(query)

In [4]:
query = '''
    select column_name from information_schema.columns
    where table_name = 'procedures' and
    table_schema = database();
'''
sql(query)
procedure_columns = [c[0] for c in sql(query).values.tolist()]
for col in procedure_columns:
    query = f'''
        alter table procedures
        rename column {col} to {col.lower()}
    '''
    run(query)

In [5]:
query = '''
    select column_name from information_schema.columns
    where table_name = 'procedures' and
    table_schema = database();
'''
sql(query)
procedure_columns = [c[0] for c in sql(query).values.tolist()]

In [6]:
query = """
UPDATE observations
SET date = STR_TO_DATE(
    TRIM(REPLACE(REPLACE(REPLACE(date, '"', ''), '\r', ''), '\n', '')),
    '%Y-%m-%dt%H:%i:%sz'
);
"""
run(query)

In [7]:
query = """
ALTER TABLE observations
MODIFY `date` date;
"""
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [8]:
for col in procedure_columns:
    query = f'''
        update procedures
        set {col} = lower({col})
    '''
    run(query)

In [10]:
query = '''
    alter table procedures
    rename column reasondescription to reason;
'''
run(query)

In [11]:
select('procedures')

,date,patient,encounter,description,base_cost,reason
0,2011-04-30t00:26:23z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,insertion of subcutaneous contraceptive,14896.56,None
1,2010-07-27t12:58:08z,10339b10-3cd1-4ac3-ac13-ec26728cb592,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,medication reconciliation (procedure),726.51,None
2,2010-11-20t03:04:34z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,7ff86631-0378-4bfc-92ce-1edd697eb18e,medication reconciliation (procedure),788.50,None
3,2011-02-07t03:04:34z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,throat culture (procedure),2070.44,acute viral pharyngitis (disorder)
4,2011-04-19t03:04:34z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,640837d9-845a-433c-9fad-47426664a69d,throat culture (procedure),2479.39,acute viral pharyngitis (disorder)


In [13]:
query = '''
    select column_name from information_schema.columns
    where table_name = 'procedures' and
    table_schema = database();
'''
sql(query)
procedure_columns = [c[0] for c in sql(query).values.tolist()]

In [14]:
for col in procedure_columns:
    query = f'''
        select sum({col} is null) as {col}_null from procedures
        having {col}_null > 0;
    '''
    display(sql(query))

,base_cost_null


,date_null


,description_null


,encounter_null


,patient_null


,reason_null
0,15544.0


In [15]:
query = '''
    update procedures
    set reason = 'unknow' where reason is null;
'''
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [16]:
select('procedures')

,date,patient,encounter,description,base_cost,reason
0,2011-04-30t00:26:23z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,insertion of subcutaneous contraceptive,14896.56,unknow
1,2010-07-27t12:58:08z,10339b10-3cd1-4ac3-ac13-ec26728cb592,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,medication reconciliation (procedure),726.51,unknow
2,2010-11-20t03:04:34z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,7ff86631-0378-4bfc-92ce-1edd697eb18e,medication reconciliation (procedure),788.50,unknow
3,2011-02-07t03:04:34z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,throat culture (procedure),2070.44,acute viral pharyngitis (disorder)
4,2011-04-19t03:04:34z,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,640837d9-845a-433c-9fad-47426664a69d,throat culture (procedure),2479.39,acute viral pharyngitis (disorder)
